hour_list 생성


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd #판다스 library import
import requests
import urllib.request
from bs4 import BeautifulSoup
 
url = "https://music.bugs.co.kr/chart"#크롤링 할 url, 벅스 100차트
hour_list=[]#hour_list 생성

# extract all hour 
html = requests.get(url)
soup = BeautifulSoup(html.content,"html.parser")
time = soup.find('ul',{'class':'list-layer-select center'}) #시간이 들어있는 ul태그와 클래스 명 검색

for li in time.find_all('li'):#for문으로 <li> 태그 속 텍스트 추출
    hours = li.find(class_='hyrend').text
    hours = hours[:2]#분을 제외한 시간만 필요하므로 뒤에 지워주기
    hour_list.insert(0,hours)#추출한 시간은 hour_list에 저장
hour_list

페이지 소스 , top_100_list 선언

In [ ]:
import urllib.request
from bs4 import BeautifulSoup


base_url = "https://music.bugs.co.kr/chart/track/realtime/total?chartdate=20200506&charthour={h}"

top_100_list = {}#top_100_list생성

In [ ]:
for hour in hour_list : 

  url = base_url.format(h = hour)#url의 {h}의 스트링을 초기화하고 hour로 변경
  html = urllib.request.urlopen(url).read()
  soup = BeautifulSoup(html, "html.parser")
  table = soup.find('table',{'class':'list trackList byChart'}) #ranking, title, artist 정보가 들어있는 테이블을 검색

  ranking_list, title_list, artist_list =  [], [], [] #ranking_list, title_list, artist_list 생성

  for tr in table.tbody.find_all('tr'):#위에서 찾은 table의 <tbody> 안에 있는 <tr>태그 모두 검색
    ranking = tr.find(class_="ranking").strong.text#ranking = class : ranking, <strong> 안의  text 
    title = tr.find(class_="title").a.text#title = class : title, <a> 안의 text
    artist = tr.find(class_="artist").a.text#artist = class : artist, <a> 안의 text
#위에서 찾은 text들을 각 list에 붙이기
    ranking_list.append(ranking)
    title_list.append(title)
    artist_list.append(artist)
#DataFrame 생성, index = 0~99, columns = ranking, title, artist
  df = pd.DataFrame(index = range(0,100), columns = ['ranking', 'title', 'artist'])
  df.ranking = ranking_list
  df.title = title_list
  df.artist = artist_list
#시간대별 top_100_list = Data 넣기
  top_100_list[hour] = df

In [ ]:
print(df)

In [ ]:
#시간대별 top_100_list 저장, 
for key in hour_list : 
    top_100_list[key].to_csv("top_100_list_{hour}.csv".format(hour=key), index = False)

artist 빈도수 

In [ ]:
def counter(input_list):
  word_count = {}
  for word in artist_list:
    if word in word_count:
      word_count[word] += 1
    else:
      word_count[word] = 1
  return word_count
word_count = counter(artist_list)

word_count = sorted(word_count.items(), key=lambda x:x[1], reverse = True)#sorted함수로 word_count 정렬
print(word_count)

[('아이유(IU)', 5), ('NCT DREAM', 5), ('오마이걸(OH MY GIRL)', 4), ('폴킴(Paul Kim)', 4), ('태연 (TAEYEON)', 2), ('장범준', 2), ('(여자)아이들', 2), ('Dua Lipa(두아 리파)', 2), ('백예린', 2), ('Anne-Marie(앤 마리)', 2), ('방탄소년단', 2), ('노을', 2), ('마크툽(Maktub)', 2), ('임영웅', 2), ('HYNN(박혜원)', 2), ('Lauv(라우브)', 2), ('영탁', 2), ('조정석', 1), ('에이프릴(APRIL)', 1), ('청하', 1), ('Apink (에이핑크)', 1), ('휘인(Whee In)', 1), ('먼데이 키즈(Monday Kiz)', 1), ('조이 (JOY)', 1), ('ITZY (있지)', 1), ('어반자카파', 1), ('가호(Gaho)', 1), ('규현 (KYUHYUN)', 1), ('권진아', 1), ('성시경', 1), ('곽진언', 1), ('창모(CHANGMO)', 1), ('Tones And I', 1), ('지코', 1), ('Conan Gray', 1), ('벤', 1), ('하현우(국카스텐)', 1), ('IZ*ONE(아이즈원)', 1), ('길구봉구', 1), ('Red Velvet (레드벨벳)', 1), ('엠씨더맥스 (M.C the MAX)', 1), ('오반', 1), ('Sam Smith(샘 스미스)', 1), ('Maroon 5(마룬 파이브)', 1), ('둘째이모 김다비', 1), ('김필(Kim Feel)', 1), ('제이레빗(J Rabbit)', 1), ('NCT 127', 1), ('솔라 (마마무)', 1), ('Etham', 1), ('잔나비', 1), ('마마무(Mamamoo)', 1), ('신예영', 1), ('AKMU (악동뮤지션)', 1), ('The Weeknd(위켄드)', 1), ('Stormzy(스톰지)', 1), ('MC몽

artist_list 워드 클라우드

>font 다운로드


In [ ]:
!apt-get update
!apt-get install fonts-nanum
# !apt-get install fontconfig
# !fc-cache -fv
# !cp /usr/share/fonts/truetype/nanum/Nanum* /usr/local/lib/python3.6/dist-packages/pytagcloud/fonts
# !rm -rf /content/.cache/pytagcloud/*

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 271 kB in 2s (118 kB/s)
Reading package lists... Done
Reading package lis

>word cloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

    #wordCloud생성
    #한글꺠지는 문제 해결하기위해 font_path 지정
font_path='/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
wc = WordCloud(font_path=font_path, 
               background_color='white', width=800, height=600)
cloud = wc.generate_from_frequencies(dict(word_count))
plt.figure(figsize=(10, 8))
plt.axis('off')
plt.imshow(cloud)

In [ ]:
!cd /usr/share/fonts/truetype/nanum | ls 

drive		     top_100_list_10.csv  top_100_list_17.csv
sample_data	     top_100_list_11.csv  top_100_list_18.csv
top_100_list_00.csv  top_100_list_12.csv  top_100_list_19.csv
top_100_list_01.csv  top_100_list_13.csv  top_100_list_20.csv
top_100_list_07.csv  top_100_list_14.csv  top_100_list_21.csv
top_100_list_08.csv  top_100_list_15.csv  top_100_list_22.csv
top_100_list_09.csv  top_100_list_16.csv  top_100_list_23.csv


image 크롤링

>chormedriver, selenium 설치

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
import sys
sys.path.insert(0, '/usr/bin/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()#chromedriver를 구동하기 위해 옵션 추가
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Fetched 163 kB in 2s (90.8 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency

In [ ]:
from urllib.request import urlretrieve#urlretrieve : url문서의 내용을 파일로 저장
import os
from selenium.webdriver.common.keys import Keys
wd = webdriver.Chrome("chromedriver", chrome_options = chrome_options)
#from bs4 import BeautifulSoup#태그 이름을 사용해서 찾음
bs = BeautifulSoup(html_doc, 'html.parser')
imageDir = "/content/drive/My Drive/Colab Notebooks/"+searchItem

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


>image 크롤링

In [ ]:
# 페이지 소스 요청
url = "https://music.bugs.co.kr/chart/track/realtime/total?chartdate=20200506&charthour=18"
req = requests.get(url)
text = req.text

soup = BeautifulSoup(text)
tr_list = soup.findAll(class_="list trackList byChart")[0].findAll("tr")
tr_list = tr_list[1:] # 테이블 헤더 제거

for tr in tr_list : 

  # 이미지 다운로드 경로
  img_source = tr.find(class_="thumbnail").img.get("src")

  # 확장자 찾기(모든 이미지의 확장자가 jpg가 아닐 수도 있으므로)
  temp = img_source[: img_source.find("?")][::-1]
  ext = temp[: temp.find(".")][::-1]

  # 이미지 파일명 (노래 제목)
  img_fname = tr.find(class_="title").a.text

  urlretrieve(img_source, os.path.join(imageDir, img_fname + "." + ext))